In [2]:
import pandas as pd, requests 
from bs4 import BeautifulSoup 
import sqlalchemy, sqlite3 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 30) 
pd.set_option('display.max_colwidth', None)

In [3]:
def clean_s(s):
    start = s.find('>')+1 
    end = s.find('/')-2
    return s[start:end]
def name_format(name):
    name = (str(name[0])
            .replace('č', 'c')
            .replace('ć', 'c')
            .replace("'", '')
    ) 
    return name 
def get_report(player):
    player= name_format(player)
    player_split = player.lower().replace('.', '').split(' ') 
    if len(player_split) == 2:
        try:
            r = requests.get(f"https://hoopshype.com/{player_split[0]}-{player_split[1]}-scouting-report/") 
        except:
            return ''
    else:
        try:
            r = requests.get(f"https://hoopshype.com/{player_split[0]}-{player_split[1]}-{player_split[2]}-scouting-report/")
        except:
            return '' 
    soup = BeautifulSoup(r.content) 
    report = soup.find('p', style="padding: 0 20px;line-height:1.4em")  
    report = clean_s(str(report)) if report is not None else '' 
    return report 

In [164]:

engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db')
players_2023_24 = pd.read_sql_query('SELECT Player FROM master_2023_24', con=engine) 

# get_report(player_names.iloc[0]['Player']) 

players_2023_24['Report'] = players_2023_24.apply(lambda player:get_report(player), axis=1) 
players_2023_24['Report']
    

C:\Users\bchan\AppData\Local\Temp\ipykernel_7108\2229652679.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = (str(name[0])


0      A bully in the frontcourt… At his best, the mo...
1      Generational talent.. His feel for the game is...
2      One of the most unique players ever… Unstoppab...
3      Versatile combo guard that can score against a...
4      A bit undersized but don’t let that fool you… ...
                             ...                        
567                                                     
568    Solid size and quickness on the wing at 6-foot...
569    6-foot-6 combo guard… Left dominant… Improving...
570                                                     
571                                                     
Name: Report, Length: 572, dtype: object

In [171]:
players_2023_24.head(30) 

,Player,Report
0,Joel Embiid,"A bully in the frontcourt… At his best, the most dominant big man in the NBA on both ends… Excellent combination of size, strength, mobility and skill… Nasty face-up game… Can knock down jumpers with defenders in his face… Also a three-point threat… Draws more fouls than anybody else and knocks down free throws at a high rate… Intimidating defender… Typically not in top-notch shape… Stamina and health are issues with him"
1,Luka Dončić,Generational talent.. His feel for the game is unmatched… Mature beyond his age… Great agility to break through defenses… Not afraid of taking big shots… Can really create for himself and others… Spectacular passing ability… Often embarrasses opponents with a great combination of ballhandling and changes of direction… Great timing to rebound… Can be too passive defensively… Going to be in MVP discussion for next decade… Complains too much to referees… Loses focus when he does this and costs his team in that moment… Can he be a locker room leader
2,Giannis Antetokounmpo,"One of the most unique players ever… Unstoppable in the open court… Finishes at the rim like few players, usually being among the league leaders in points in the paint… Freakish length and athleticism… Great handles for his size… His wingspan and instincts make him a great defender… Still hungry to improve his game… Has made strides every year… His jumper has improved, but will not be mistaken for a shooter… Below-average free-throw shooter… A rare low-maintenance superstar"
3,Shai Gilgeous-Alexander,Versatile combo guard that can score against anyone… Length gives him an advantage against other wings… Solid body control to attack the rim… Can get to the hoop at will… Scores a massive amount of points in the paint… Excellent from the mid-range… Can play on and off the ball… Good defender on the perimeter… Good timing to rebound and block… Has emerged as a star… Has the ability to hit the big shot… Doesn’t take many threes
4,Jalen Brunson,A bit undersized but don’t let that fool you… Can get to the rim at an elite level… Crafty and relies on his footwork to be effective… Has the ability to hit the stepback three as well… Playmaker that can set teammates up for open shots… Really has improved every season he’s been in the league… Borderline All-Star point guard… Small frame leaves him somewhat bad defensively… At least he gives effort
5,Devin Booker,"A throwback bucket-getting shooting guard who is a three-level scorer… One of the most confident scorers in the league… Big shot maker down the stretch… Can put up 25 in his sleep… Much-improved defensively, displaying much more effort on that end… Beast in the midrange with a strong pull-up game… Solid playmaker… Still has room to improve his game"
6,Kevin Durant,Arguably the best shot creator ever for his size… Makes scoring look easy… Excellent agility and body control… Almost unstoppable in fast-break situations… Elite spot-up three-point shooter… Active defender… Great wingspan and timing protecting the rim… Can have some tunnel vision when attacking the rim
7,Jayson Tatum,"Versatile forward with plenty of ways to score… Nasty face-up game with an explosive first step and ability to glide through defenses… Very good athleticism… Pivoted away from taking mid-range jumpers to threes… Percentages are not awesome, though… Great instincts on defense… Drawing more fouls and knocks down free throws at a high rate… Good size and lateral quickness defensively… Emerged as a legitimate superstar… May win an MVP before his career is over"
8,Donovan Mitchell,"Huge offensive talent… One of the most gifted scorers of his generation… Great ability to drive to the rim… Can easily assume a playmaker role… Efficient spot-up shooter… Energetic defensive player… Loves clutch time… Very inconsistent from game to game… Poor defender, but has the physical attributes to be much better… A bit of a ball stopper… Needs to improve more on being a playmaker for teammates"
9,De'Aar

In [ ]:
reports_2023_24.to_sql('reports_2023_24', con=engine, index=False, if_exists='replace') 

NameError: name 'players_2023_24' is not defined

In [68]:
engine = 'sqlite:///../../DB/ballbase.db' 
reports_2023_24 = pd.read_sql_query('SELECT * FROM reports_2023_24', con=engine) 

reports_2023_24['Report'] = reports_2023_24['Report'].str.lower() 
# reports_2023_24['Report'] = reports_2023_24['Report'].str.replace('[…,]', ' ', regex=True)
reports_2023_24['Report'] = reports_2023_24['Report'].str.replace(r'[^a-zA-z\s]', ' ', regex=True)
reports_2023_24['Report'] = reports_2023_24['Report'].str.strip().replace(r'\s', ' ', regex=True) 

import nltk 
nltk.download('stopwords') 
stop_words = nltk.corpus.stopwords.words('english') 
reports_2023_24['Report'] = reports_2023_24['Report'].apply(lambda des: ' '.join([word for word in des.split() if word not in stop_words])) 

reports_2023_24 




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bchan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Player,Report
0,Joel Embiid,bully frontcourt best dominant big man nba ends excellent combination size strength mobility skill nasty face game knock jumpers defenders face also three point threat draws fouls anybody else knocks free throws high rate intimidating defender typically top notch shape stamina health issues
1,Luka Dončić,generational talent feel game unmatched mature beyond age great agility break defenses afraid taking big shots really create others spectacular passing ability often embarrasses opponents great combination ballhandling changes direction great timing rebound passive defensively going mvp discussion next decade complains much referees loses focus costs team moment locker room leader
2,Giannis Antetokounmpo,one unique players ever unstoppable open court finishes rim like players usually among league leaders points paint freakish length athleticism great handles size wingspan instincts make great defender still hungry improve game made strides every year jumper improved mistaken shooter average free throw shooter rare low maintenance superstar
3,Shai Gilgeous-Alexander,versatile combo guard score anyone length gives advantage wings solid body control attack rim get hoop scores massive amount points paint excellent mid range play ball good defender perimeter good timing rebound block emerged star ability hit big shot take many threes
4,Jalen Brunson,bit undersized let fool get rim elite level crafty relies footwork effective ability hit stepback three well playmaker set teammates open shots really improved every season league borderline star point guard small frame leaves somewhat bad defensively least gives effort
...,...,...
567,Javonte Smart,
568,Justin Jackson,solid size quickness wing foot best skill driving taking couple dribbles attacking floater range solid teardrop shot despite solid shooting stroke numbers outside bad throughout career decent defender plays energy expected much scoring
569,Malcolm Cazalon,foot combo guard left dominant improving finisher around rim solid shooter although shooting mechanics inconsistent likes play transition may trouble speed nba guard bigger players post needs work going right bit reactionary pump fakes players blow often
570,Dmytro Skapintsev,


In [ ]:
import tensorflow as tf, keras, nltk  